In [1]:
!pip install kornia
!pip install torch==1.5.1+cu101  -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch_lightning    

     |████████████████████████████████| 163kB 9.5MB/s 
     |████████████████████████████████| 752.0MB 23kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.5.0 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 10kB/s 
ERROR: kornia 0.3.1 has requirement torch==1.5.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.5.0
    Uninstalling torch-1.5.0:
      Successfully uninstalled torch-1.5.0
     |████████████████████████████████| 317kB 6.5MB/s 
     |████████████████████████████████| 276kB 9.4MB/s 
     |████████████████████████████████| 829kB 14.5MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=7f266ec5107c505ccf2642f9bdd67a811

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import os
import argparse
import multiprocessing
import torch

from pathlib import Path
from PIL import Image

from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import sys
sys.path.append('/content/drive/My Drive/hateful_memes_data/')
from byol_pytorch import BYOL

In [2]:
resnet = models.resnet18(pretrained=True)
BATCH_SIZE = 32
EPOCHS     = 100
LR         = 3e-4
NUM_GPUS   = 1
IMAGE_SIZE = 256
IMAGE_EXTS = ['.png']
NUM_WORKERS = multiprocessing.cpu_count()

# pytorch lightning module

class SelfSupervisedLearner(pl.LightningModule):
    def __init__(self, net, **kwargs):
        super().__init__()
        self.learner = BYOL(net, **kwargs)

    def forward(self, images):
        return self.learner(images)

    def training_step(self, images, _):
        loss = self.forward(images)
        return {'loss': loss}

    def validation_step(self, images, _):
        loss = self.forward(images)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss)
        return result

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=LR)

    def on_before_zero_grad(self, _):
        self.learner.update_moving_average()

# images dataset

def expand_greyscale(t):
    return t.expand(3, -1, -1)

class ImagesDataset(Dataset):
    def __init__(self, folder, image_size):
        super().__init__()
        self.folder = folder
        self.paths = []
        print(folder)

        for path in Path(f'{folder}').glob('**/*'):
            _, ext = os.path.splitext(path)
            if ext.lower() in IMAGE_EXTS:
                self.paths.append(path)

        print(f'{len(self.paths)} images found')

        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Lambda(expand_greyscale)
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path)
        img = img.convert('RGB')
        return self.transform(img)

In [3]:
train_ds = ImagesDataset('/content/drive/My Drive/hateful_memes_data/train/data', IMAGE_SIZE)
val_ds = train_ds = ImagesDataset('/content/drive/My Drive/hateful_memes_data/val/data', IMAGE_SIZE)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

model = SelfSupervisedLearner(
    resnet,
    image_size = IMAGE_SIZE,
    hidden_layer = 'avgpool',
    projection_size = 256,
    projection_hidden_size = 4096,
    moving_average_decay = 0.99
)

checkpoint_callback = ModelCheckpoint(filepath='/content/drive/My Drive/hateful_memes_data/resnet18_weights_{epoch}',
                                      monitor='val_loss', save_top_k=3, period=3)

trainer = pl.Trainer(gpus=NUM_GPUS, max_epochs=EPOCHS, checkpoint_callback=checkpoint_callback)
trainer.fit(model, train_loader, val_loader)

/content/drive/My Drive/hateful_memes_data/train/data
8500 images found
/content/drive/My Drive/hateful_memes_data/val/data
500 images found


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type | Params
---------------------------------
0 | learner | BYOL | 31 M  


1

In [9]:
loaded_model = model.load_from_checkpoint(checkpoint_path='resnet18_model_checkpoint_epoch_100.ckpt')

TypeError: ignored